In [3]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import joblib
from sklearn.preprocessing import StandardScaler
import shutil
import dlib

loaded_knn_model = joblib.load('knn_model.joblib')
detector = dlib.get_frontal_face_detector()

def preprocess_face_image(face_img):
    resized_img = cv2.resize(face_img, (224, 224))
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    normalized_img = gray_img / 255.0
    return normalized_img

def load_training_data(data_dir):
    X_train = []
    y_train = []
    categories = os.listdir(data_dir)

    for category in categories:
        category_path = os.path.join(data_dir, category)

        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path)
            img_processed = preprocess_face_image(img)
            X_train.append(img_processed)
            y_train.append(category)

    return np.array(X_train), np.array(y_train)

data_dir = 'dataset'
X_train, y_train = load_training_data(data_dir)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

test_dataset_folder = 'test_dataset'
os.makedirs(test_dataset_folder, exist_ok=True)

knn_predicted_true_folder = os.path.join(test_dataset_folder, 'knn_predicted_true')
knn_predicted_fake_folder = os.path.join(test_dataset_folder, 'knn_predicted_fake')
os.makedirs(knn_predicted_true_folder, exist_ok=True)
os.makedirs(knn_predicted_fake_folder, exist_ok=True)

max_frames_per_video = 8
test_videos_folder = 'test_videos'

total_videos = len(os.listdir(test_videos_folder))
progress_bar = tqdm(total=total_videos, desc='Processing Test Videos')

for video_filename in os.listdir(test_videos_folder):
    video_path = os.path.join(test_videos_folder, video_filename)

    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames_with_faces = 0
    faces_labels = []

    for frame_id in range(frame_count):
        ret, frame = cap.read()
        if not ret or frames_with_faces >= max_frames_per_video:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        if faces:
            frames_with_faces += 1
            for face_id, face in enumerate(faces):
                x, y, w, h = face.left(), face.top(), face.width(), face.height()
                face_img = frame[y:y+h, x:x+w]

                if not face_img.size == 0:
                    face_img_processed = preprocess_face_image(face_img)
                    face_img_scaled = scaler.transform(face_img_processed.reshape(1, -1))
                    knn_prediction = loaded_knn_model.predict(face_img_scaled)
                    faces_labels.append(knn_prediction)

    overall_prediction = max(set(faces_labels), key=faces_labels.count)
    knn_predicted_folder = knn_predicted_true_folder if overall_prediction == 'REAL' else knn_predicted_fake_folder
    shutil.copy(video_path, os.path.join(knn_predicted_folder, video_filename))

    cap.release()
    progress_bar.update(1)

progress_bar.close()
print("Test Data Processing complete.")


Processing Test Videos:   0%|                            | 0/400 [00:00<?, ?it/s]

ValueError: X has 50176 features, but KNeighborsClassifier is expecting 512 features as input.